In [1]:
import pandas as pd
import json
from sodapy import Socrata
import os
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
from config2 import app_token
from geojson import Feature, FeatureCollection, Point

In [2]:
# ## PANDAS TO GEOJSON FUNCTION
# def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
#     geojson = {'type':'FeatureCollection', 'features':[]}
#     for _, row in df.iterrows():
#         feature = {'type':'Feature',
#                    'properties':{},
#                    'geometry':{'type':'Point',
#                                'coordinates':[]}}
#         feature['geometry']['coordinates'] = [row[lon],row[lat]]
#         for prop in properties:
#             feature['properties'][prop] = row[prop]
#         geojson['features'].append(feature)
#     return geojson

In [3]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", none)

# Example authenticated client (needed for non-public datasets):
# client = Socrata("data.cityofchicago.org",
#                  "du5ugsgl0w1g4ql37x78pjbc5",
#                  username= "dr.harmony",
#                  password= "F3ar&Loathing")

# # Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofchicago.org",
                 app_token)
#                  username= username,
#                  password= password)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("85ca-t3if", limit = 500000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [4]:
results_df

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,statements_taken_i,photos_taken_i,work_zone_i,work_zone_type,workers_present_i,dooring_i,rd_no,lane_cnt
0,af1c6f0ae0dfa8824e8f84f516136209989fb974edfbdc...,2020-10-14T00:33:00.000,30,STOP SIGN/FLASHER,OTHER,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,595d965d96ec041ec8736e519f69e0499ca5aa79332de3...,2020-10-14T00:13:00.000,25,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1e71e0b6362d503e05fafc0b73db3eeb202dbf93c9a3f0...,2020-10-13T23:42:00.000,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b5187c75caa5e2fa899c2f95d60a77bf713532c806cae4...,2020-10-13T22:41:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,CENTER TURN LANE,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42a59bf2b5eb302ab2ebfff40341c7113b7bd70a2acf2d...,2020-10-13T22:00:00.000,30,UNKNOWN,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447367,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
447368,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
447369,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
447370,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [182]:
## CLEAN UP CRASHES DF ** DELETE NaN values in beats, lat/long, remove lat/long with 0 value ** this is key for the MERGE later
results_nadrop_df = results_df.dropna(subset = ['beat_of_occurrence', 'latitude', 'longitude'])
# results_df = results_df.dropna(subset = ['latitude'])
# results_df = results_df.dropna(subset = ['longitude'])
# results_nazero_df = results_nadrop_df[results_nadrop_df.latitude != 0]
# results_nazero_df = results_nazero_df[results_nazero_df.longitude != 0]
results_clean = results_nadrop_df.drop_duplicates(subset='crash_record_id', keep="first")
results_clean

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,statements_taken_i,photos_taken_i,work_zone_i,work_zone_type,workers_present_i,dooring_i,rd_no,lane_cnt
0,af1c6f0ae0dfa8824e8f84f516136209989fb974edfbdc...,2020-10-14T00:33:00.000,30,STOP SIGN/FLASHER,OTHER,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,595d965d96ec041ec8736e519f69e0499ca5aa79332de3...,2020-10-14T00:13:00.000,25,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1e71e0b6362d503e05fafc0b73db3eeb202dbf93c9a3f0...,2020-10-13T23:42:00.000,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b5187c75caa5e2fa899c2f95d60a77bf713532c806cae4...,2020-10-13T22:41:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,CENTER TURN LANE,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42a59bf2b5eb302ab2ebfff40341c7113b7bd70a2acf2d...,2020-10-13T22:00:00.000,30,UNKNOWN,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447367,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
447368,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
447369,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
447370,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [191]:
## DROP ZEROS FROM LAT IF PRESENT
results_drop_latlong_zeros = results_clean[ (results_clean['latitude'] == 0) & (results_clean['longitude'] == '0')].index 
  
# drop these given row 
# indexes from dataFrame 
results_clean.drop(results_drop_latlong_zeros, inplace = True) 
  
results_clean

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,statements_taken_i,photos_taken_i,work_zone_i,work_zone_type,workers_present_i,dooring_i,rd_no,lane_cnt
0,af1c6f0ae0dfa8824e8f84f516136209989fb974edfbdc...,2020-10-14T00:33:00.000,30,STOP SIGN/FLASHER,OTHER,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,595d965d96ec041ec8736e519f69e0499ca5aa79332de3...,2020-10-14T00:13:00.000,25,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1e71e0b6362d503e05fafc0b73db3eeb202dbf93c9a3f0...,2020-10-13T23:42:00.000,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b5187c75caa5e2fa899c2f95d60a77bf713532c806cae4...,2020-10-13T22:41:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,CENTER TURN LANE,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42a59bf2b5eb302ab2ebfff40341c7113b7bd70a2acf2d...,2020-10-13T22:00:00.000,30,UNKNOWN,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447367,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
447368,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
447369,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
447370,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [192]:
results_clean.reset_index(drop=True)

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,statements_taken_i,photos_taken_i,work_zone_i,work_zone_type,workers_present_i,dooring_i,rd_no,lane_cnt
0,af1c6f0ae0dfa8824e8f84f516136209989fb974edfbdc...,2020-10-14T00:33:00.000,30,STOP SIGN/FLASHER,OTHER,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,595d965d96ec041ec8736e519f69e0499ca5aa79332de3...,2020-10-14T00:13:00.000,25,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1e71e0b6362d503e05fafc0b73db3eeb202dbf93c9a3f0...,2020-10-13T23:42:00.000,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",ANGLE,FOUR WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b5187c75caa5e2fa899c2f95d60a77bf713532c806cae4...,2020-10-13T22:41:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,CENTER TURN LANE,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42a59bf2b5eb302ab2ebfff40341c7113b7bd70a2acf2d...,2020-10-13T22:00:00.000,30,UNKNOWN,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444938,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
444939,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
444940,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
444941,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [193]:
## Change 'crash_date' to Datetime data type
results_clean['crash_date'] = pd.to_datetime(results_clean['crash_date'])

In [195]:
## FILTER OUT A YEAR (2019)
results_df2019 = results_clean[results_clean['crash_date'].dt.year == 2019].reset_index(drop=True)
results_df2019

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,private_property_i,statements_taken_i,photos_taken_i,work_zone_i,work_zone_type,workers_present_i,dooring_i,rd_no,lane_cnt
0,032ca7e09951de4385eec78a50a5f9f33b217838578b29...,2019-12-31 23:58:00,35,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT ON GRADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100044,NaN
1,71065ad347ee197080534e112b27c8a5b6fb8d9bbde2c3...,2019-12-31 23:57:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100010,NaN
2,9c2724305c03bf461d72cbcf2879c96b86ed5e180b62be...,2019-12-31 23:50:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100008,NaN
3,3187e5839fab7ff654383ba44a2e5b430c856b3fa10dab...,2019-12-31 23:46:00,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100006,NaN
4,449bc7511dba6e6d605411276dc08dbeeccff39b074db0...,2019-12-31 23:45:00,25,STOP SIGN/FLASHER,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JD100842,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116824,cb22668bf69d9c99e50c615a9ff996328190859a0e8351...,2019-01-01 00:07:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC100171,1
116825,caa1629056b16f2e6ec6797d6252d6da8ec1142c0ee982...,2019-01-01 00:04:00,20,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,ONE-WAY,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC100033,2
116826,0777b210b7a60ac1338d90e20520d421c3241971049303...,2019-01-01 00:01:00,30,NO CONTROLS,NO CONTROLS,UNKNOWN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC100780,NaN
116827,cb2b9a910babe20dfad7fc27c04c29e7b199fa94dc57bf...,2019-01-01 00:00:00,35,NO CONTROLS,NO CONTROLS,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC111663,2


In [196]:
## DROP UNWANTED COLUMNS
crashes = results_df2019.drop(columns=['crash_record_id', 'location',
                                       'injuries_unknown', 'injuries_incapacitating', 'injuries_no_indication', 'injuries_reported_not_evident',
                                       'injuries_non_incapacitating', 'private_property_i', 'crash_date_est_i', 'statements_taken_i', 'photos_taken_i',
                                       'work_zone_i', 'work_zone_type', 'workers_present_i', 'dooring_i'])

In [197]:
crashes.rename(columns={"beat_of_occurrence": "Beat"}, inplace = True)

In [198]:
crashes.reset_index(drop=True)

,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,...,injuries_fatal,crash_hour,crash_day_of_week,crash_month,latitude,longitude,hit_and_run_i,intersection_related_i,rd_no,lane_cnt
0,2019-12-31 23:58:00,35,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT ON GRADE,SNOW OR SLUSH,...,0,23,3,12,41.871797864,-87.696189628,Y,NaN,JD100044,NaN
1,2019-12-31 23:57:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,...,0,23,3,12,41.812437437,-87.743434278,NaN,Y,JD100010,NaN
2,2019-12-31 23:50:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,...,NaN,23,3,12,41.887804621,-87.765268498,Y,NaN,JD100008,NaN
3,2019-12-31 23:46:00,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,...,0,23,3,12,41.910943878,-87.626392797,NaN,NaN,JD100006,NaN
4,2019-12-31 23:45:00,25,STOP SIGN/FLASHER,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,...,0,23,3,12,41.883961501,-87.674114078,Y,Y,JD100842,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116824,2019-01-01 00:07:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,0,0,3,1,41.954529286,-87.784311465,Y,NaN,JC100171,1
116825,2019-01-01 00:04:00,20,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,ONE-WAY,STRAIGHT AND LEVEL,WET,...,0,0,3,1,41.882058105,-87.63347836,NaN,Y,JC100033,2
116826,2019-01-01 00:01:00,30,NO CONTROLS,NO CONTROLS,UNKNOWN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,UNKNOWN,...,0,0,3,1,41.94685643,-87.670809132,Y,NaN,JC100780,NaN
116827,2019-01-01 00:00:00,35,NO CONTROLS,NO CONTROLS,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,...,0,0,3,1,41.793502271,-87.586406738,Y,NaN,JC111663,2


In [212]:
crashes

,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,...,injuries_fatal,crash_hour,crash_day_of_week,crash_month,latitude,longitude,hit_and_run_i,intersection_related_i,rd_no,lane_cnt
0,2019-12-31 23:58:00,35,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT ON GRADE,SNOW OR SLUSH,...,0,23,3,12,41.871797864,-87.696189628,Y,NaN,JD100044,NaN
1,2019-12-31 23:57:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,...,0,23,3,12,41.812437437,-87.743434278,NaN,Y,JD100010,NaN
2,2019-12-31 23:50:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,...,NaN,23,3,12,41.887804621,-87.765268498,Y,NaN,JD100008,NaN
3,2019-12-31 23:46:00,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,...,0,23,3,12,41.910943878,-87.626392797,NaN,NaN,JD100006,NaN
4,2019-12-31 23:45:00,25,STOP SIGN/FLASHER,UNKNOWN,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,...,0,23,3,12,41.883961501,-87.674114078,Y,Y,JD100842,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116824,2019-01-01 00:07:00,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,...,0,0,3,1,41.954529286,-87.784311465,Y,NaN,JC100171,1
116825,2019-01-01 00:04:00,20,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,ONE-WAY,STRAIGHT AND LEVEL,WET,...,0,0,3,1,41.882058105,-87.63347836,NaN,Y,JC100033,2
116826,2019-01-01 00:01:00,30,NO CONTROLS,NO CONTROLS,UNKNOWN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,UNKNOWN,...,0,0,3,1,41.94685643,-87.670809132,Y,NaN,JC100780,NaN
116827,2019-01-01 00:00:00,35,NO CONTROLS,NO CONTROLS,UNKNOWN,UNKNOWN,PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,...,0,0,3,1,41.793502271,-87.586406738,Y,NaN,JC111663,2


In [213]:

crashes['Beat'] = pd.to_numeric(crashes['Beat'])
crashes.dtypes


crash_date                 datetime64[ns]
posted_speed_limit                 object
traffic_control_device             object
device_condition                   object
weather_condition                  object
lighting_condition                 object
first_crash_type                   object
trafficway_type                    object
alignment                          object
roadway_surface_cond               object
road_defect                        object
report_type                        object
crash_type                         object
damage                             object
date_police_notified               object
prim_contributory_cause            object
sec_contributory_cause             object
street_no                          object
street_direction                   object
street_name                        object
Beat                                int64
num_units                          object
most_severe_injury                 object
injuries_total                    

In [214]:
crashes.columns


Index(['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'damage', 'date_police_notified', 'prim_contributory_cause',
       'sec_contributory_cause', 'street_no', 'street_direction',
       'street_name', 'Beat', 'num_units', 'most_severe_injury',
       'injuries_total', 'injuries_fatal', 'crash_hour', 'crash_day_of_week',
       'crash_month', 'latitude', 'longitude', 'hit_and_run_i',
       'intersection_related_i', 'rd_no', 'lane_cnt'],
      dtype='object')

In [215]:
# ## EXPORT DATA PULL FOR BU
# crashes.to_csv('accident_data_BU.txt', sep='\t', index=False)

In [217]:
## CREATING CRASHES AS GeoDataFrame
gdf_crashes2019 = gpd.GeoDataFrame(crashes, geometry=gpd.points_from_xy(crashes.longitude, crashes.latitude))

In [218]:
## EXPORT CRASHES AS JSON FILE
gdf_crashes2019.to_file("crashes_in_2019.geojson", driver='GeoJSON')

In [204]:
## FOR EXPORT TO NEIGHBORHOOD PARSER SCRIPT ****
forparse = crashes.drop(columns=['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'damage', 'date_police_notified', 'prim_contributory_cause',
       'sec_contributory_cause', 'street_no', 'street_direction',
       'street_name', 'Beat', 'num_units', 'most_severe_injury',
       'injuries_total', 'injuries_fatal', 'crash_hour', 'crash_day_of_week',
       'crash_month','hit_and_run_i',
       'intersection_related_i', 'rd_no', 'lane_cnt'])
forparse.shape

(116829, 2)

In [205]:
# ## EXPORTING LIST OF LAT LONG

# forparse.to_csv('forparse.txt', sep='\t', index=False)

In [135]:
## READ PARSED LAT LONG WITH CHICAGO NEIGHBORHOOD
neighbor_lat_lon = pd.read_csv("static/data/parsed_community_with_polygon.csv")
neighbor_lat_lon

,latitude,longitude,Polygon Neighborhood
0,41.871798,-87.696190,GARFIELD PARK
1,41.812437,-87.743434,MIDWAY AIRPORT
2,41.887805,-87.765269,AUSTIN
3,41.910944,-87.626393,GOLD COAST
4,41.883961,-87.674114,UNITED CENTER
...,...,...,...
116824,41.954529,-87.784311,PORTAGE PARK
116825,41.882058,-87.633478,LOOP
116826,41.946856,-87.670809,LAKE VIEW
116827,41.793502,-87.586407,HYDE PARK


In [136]:
## Split Neighborhood Name to 1 name after ','
neighbor_latlon_comname = neighbor_lat_lon.rename(columns={"Polygon Neighborhood": "communityArea"})
neighbor_latlon_comname

,latitude,longitude,communityArea
0,41.871798,-87.696190,GARFIELD PARK
1,41.812437,-87.743434,MIDWAY AIRPORT
2,41.887805,-87.765269,AUSTIN
3,41.910944,-87.626393,GOLD COAST
4,41.883961,-87.674114,UNITED CENTER
...,...,...,...
116824,41.954529,-87.784311,PORTAGE PARK
116825,41.882058,-87.633478,LOOP
116826,41.946856,-87.670809,LAKE VIEW
116827,41.793502,-87.586407,HYDE PARK


In [150]:
neighbor_latlon_comname['communityArea'] = neighbor_latlon_comname['communityArea'].str.split(',').str[0]
neighbor_latlon_comname

,latitude,longitude,communityArea
0,41.871798,-87.696190,GARFIELD PARK
1,41.812437,-87.743434,MIDWAY AIRPORT
2,41.887805,-87.765269,AUSTIN
3,41.910944,-87.626393,GOLD COAST
4,41.883961,-87.674114,UNITED CENTER
...,...,...,...
116824,41.954529,-87.784311,PORTAGE PARK
116825,41.882058,-87.633478,LOOP
116826,41.946856,-87.670809,LAKE VIEW
116827,41.793502,-87.586407,HYDE PARK


In [165]:
neighbor_latlon_comname.communityArea.unique()

array(['GARFIELD PARK', 'MIDWAY AIRPORT', 'AUSTIN', 'GOLD COAST',
       'UNITED CENTER', 'BRIDGEPORT', 'RIVER NORTH', nan,
       'BACK OF THE YARDS', 'NORTH LAWNDALE', 'BUCKTOWN', 'NORTH PARK',
       'NEAR SOUTH SIDE', 'WEST LOOP', 'ROGERS PARK', 'MUSEUM CAMPUS',
       'MARQUETTE PARK', 'BELMONT CRAIGIN', 'RIVERDALE', 'DUNNING',
       'HUMBOLDT PARK', 'SOUTH SHORE', 'AUBURN GRESHAM', 'CHATHAM',
       'UPTOWN', 'WEST RIDGE', 'EDGEWATER', 'ENGLEWOOD', 'IRVING PARK',
       'KENWOOD', 'LAKE VIEW', 'LINCOLN PARK', 'WASHINGTON HEIGHTS',
       'WASHINGTON PARK', 'LITTLE VILLAGE', 'BRONZEVILLE', 'NORWOOD PARK',
       'SOUTHEAST SIDE', 'PULLMAN', 'WOODLAWN', 'LITTLE ITALY',
       'BELMONT CRAGIN', 'WEST LAWN', 'AVALON PARK', 'NORTH CENTER',
       'ARCHER HEIGHTS', 'WICKER PARK', 'SOUTH CHICAGO', 'LOOP',
       'LOGAN SQUARE', 'BEVERLY', 'BRIGHTON PARK', 'PORTAGE PARK',
       'ASHBURN', 'WEST PULLMAN', 'STREETERVILLE', 'LINCOLN SQUARE',
       'MONTCLARE', 'MOUNT GREENWOOD', 'LOWER W

In [166]:
neighbor_latlon_comname["communityArea"].replace({"MIDWAY AIRPORT": "GARFIELD RIDGE", "GARFIELD PARK": "WEST & EAST GARFIELD PARK",
                                                 "RUSH & DIVISION": "NEAR NORTH SIDE"}, inplace=True)

In [167]:
## READ Chicago Area geoJSON file for merge of Just Community Areas
chicagoAreas_df = gpd.read_file("static/data/chicago-community-areas.geojson")

In [168]:
chicagoAreas_df.community.unique()


array(['DOUGLAS', 'OAKLAND', 'FULLER PARK', 'GRAND BOULEVARD', 'KENWOOD',
       'LINCOLN SQUARE', 'WASHINGTON PARK', 'HYDE PARK', 'WOODLAWN',
       'ROGERS PARK', 'JEFFERSON PARK', 'FOREST GLEN', 'NORTH PARK',
       'ALBANY PARK', 'PORTAGE PARK', 'IRVING PARK', 'DUNNING',
       'MONTCLARE', 'BELMONT CRAGIN', 'WEST RIDGE', 'HERMOSA', 'AVONDALE',
       'LOGAN SQUARE', 'HUMBOLDT PARK', 'WEST TOWN', 'AUSTIN',
       'WEST GARFIELD PARK', 'EAST GARFIELD PARK', 'NEAR WEST SIDE',
       'NORTH LAWNDALE', 'UPTOWN', 'SOUTH LAWNDALE', 'LOWER WEST SIDE',
       'NEAR SOUTH SIDE', 'ARMOUR SQUARE', 'NORWOOD PARK',
       'NEAR NORTH SIDE', 'LOOP', 'SOUTH SHORE', 'CHATHAM', 'AVALON PARK',
       'SOUTH CHICAGO', 'BURNSIDE', 'CALUMET HEIGHTS', 'ROSELAND',
       'NORTH CENTER', 'PULLMAN', 'SOUTH DEERING', 'EAST SIDE',
       'WEST PULLMAN', 'RIVERDALE', 'HEGEWISCH', 'GARFIELD RIDGE',
       'ARCHER HEIGHTS', 'BRIGHTON PARK', 'MCKINLEY PARK', 'LAKE VIEW',
       'BRIDGEPORT', 'NEW CITY', 'WEST EL

In [169]:
## Remove all columns but the ones needed.
chicagoAreas_df_clean = chicagoAreas_df.drop(columns=['area', 'shape_area', 'perimeter', 'area_num_1',
       'comarea_id', 'comarea', 'shape_len', 'geometry'])
chicagoAreas_df_clean

,community,area_numbe
0,DOUGLAS,35
1,OAKLAND,36
2,FULLER PARK,37
3,GRAND BOULEVARD,38
4,KENWOOD,39
...,...,...
72,MOUNT GREENWOOD,74
73,MORGAN PARK,75
74,OHARE,76
75,EDGEWATER,77


In [170]:
## Make sure Chicago Area name is changed for merge and column rename.
chicagoAreas_df_clean['area_numbe'] = pd.to_numeric(chicagoAreas_df_clean['area_numbe'])
chicagoAreas_df_clean

,community,area_numbe
0,DOUGLAS,35
1,OAKLAND,36
2,FULLER PARK,37
3,GRAND BOULEVARD,38
4,KENWOOD,39
...,...,...
72,MOUNT GREENWOOD,74
73,MORGAN PARK,75
74,OHARE,76
75,EDGEWATER,77


In [171]:
## Column name change
chicago_areas_namechange = chicagoAreas_df_clean.rename(columns={"area_numbe": "community_num"})
pd.set_option("display.max_rows", None, "display.max_columns", None)
chicago_areas_namechange

,community,community_num
0,DOUGLAS,35
1,OAKLAND,36
2,FULLER PARK,37
3,GRAND BOULEVARD,38
4,KENWOOD,39
5,LINCOLN SQUARE,4
6,WASHINGTON PARK,40
7,HYDE PARK,41
8,WOODLAWN,42
9,ROGERS PARK,1


In [172]:
chicagoAreas_namechange = chicago_areas_namechange.rename(columns={"community": "communityArea"})
chicagoAreas_namechange

,communityArea,community_num
0,DOUGLAS,35
1,OAKLAND,36
2,FULLER PARK,37
3,GRAND BOULEVARD,38
4,KENWOOD,39
5,LINCOLN SQUARE,4
6,WASHINGTON PARK,40
7,HYDE PARK,41
8,WOODLAWN,42
9,ROGERS PARK,1


In [173]:
# def generate_id(s):
#     return abs(hash(s)) % (10 ** 10)

In [174]:
# chicagoAreas_namechange['id'] = chicagoAreas_namechange['communityArea'].apply(generate_id)
# chicagoAreas_namechange

In [175]:
chicagoAreas_namechange.dtypes

communityArea    object
community_num     int64
dtype: object

In [176]:
## *** THE MERGE TO GET COMMUNITY AREA NAME AND NUMBER ***
# crash_neighborhood_merge = pd.merge(left=neighbor_latlon_comname, right=chicagoAreas_namechange, how='left', on=['communityArea'])
crash_neighborhood_merge = neighbor_latlon_comname.merge(chicagoAreas_namechange, how='left', on=['communityArea'])

In [177]:
pd.reset_option('^display.', silent=True)
crash_neighborhood_merge

,latitude,longitude,communityArea,community_num
0,41.871798,-87.696190,WEST & EAST GARFIELD PARK,NaN
1,41.812437,-87.743434,GARFIELD RIDGE,56.0
2,41.887805,-87.765269,AUSTIN,25.0
3,41.910944,-87.626393,GOLD COAST,NaN
4,41.883961,-87.674114,UNITED CENTER,NaN
...,...,...,...,...
116824,41.954529,-87.784311,PORTAGE PARK,15.0
116825,41.882058,-87.633478,LOOP,32.0
116826,41.946856,-87.670809,LAKE VIEW,6.0
116827,41.793502,-87.586407,HYDE PARK,41.0


In [178]:
## FILL NaN
latlon_ChiAreas = crash_neighborhood_merge.set_index("communityArea").community_num.fillna(chicagoAreas_namechange.set_index("communityArea").community_num).reset_index()
latlon_ChiAreas

,communityArea,community_num
0,WEST & EAST GARFIELD PARK,NaN
1,GARFIELD RIDGE,56.0
2,AUSTIN,25.0
3,GOLD COAST,NaN
4,UNITED CENTER,NaN
...,...,...
116824,PORTAGE PARK,15.0
116825,LOOP,32.0
116826,LAKE VIEW,6.0
116827,HYDE PARK,41.0


In [179]:
# crash_neighborhood_merge = pd.concat([crashes, neighborhoods_df], axis=1)
crash_neighborhood_merge_clean = crash_neighborhood_merge[['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment', 'injuries_fatal',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'intersection_related_i', 'damage', 'date_police_notified',
       'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
       'street_direction', 'street_name', 'District', 'Community Area', 'Beat', 'num_units',
       'most_severe_injury', 'injuries_total', 'crash_hour',
       'crash_day_of_week', 'crash_month', 'latitude', 'longitude',
       'hit_and_run_i', 'rd_no', 'lane_cnt']]
crash_neighborhood_merge_clean

KeyError: "['crash_month', 'injuries_total', 'Community Area', 'traffic_control_device', 'report_type', 'lane_cnt', 'crash_day_of_week', 'damage', 'num_units', 'device_condition', 'alignment', 'District', 'posted_speed_limit', 'date_police_notified', 'most_severe_injury', 'trafficway_type', 'roadway_surface_cond', 'street_name', 'Beat', 'crash_type', 'rd_no', 'crash_hour', 'street_no', 'intersection_related_i', 'road_defect', 'sec_contributory_cause', 'hit_and_run_i', 'weather_condition', 'first_crash_type', 'injuries_fatal', 'prim_contributory_cause', 'lighting_condition', 'crash_date', 'street_direction'] not in index"

In [180]:
crash_neighborhood_merge_clean.columns

NameError: name 'crash_neighborhood_merge_clean' is not defined

In [ ]:
crash_neighborhood_merge_clean

In [ ]:
## DROPPING COLUMNS
crashes_in_areas = crash_neighborhood_merge_clean.drop(columns=['posted_speed_limit', 'traffic_control_device', 'crash_type', 'device_condition', 'weather_condition', 'lighting_condition', 'trafficway_type', 'alignment',
'roadway_surface_cond', 'road_defect', 'report_type','intersection_related_i', 'damage', 'date_police_notified',
'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
'street_direction', 'street_name', 'District', 'num_units', 'most_severe_injury', 'crash_hour',
'crash_day_of_week', 'crash_month', 'rd_no', 'lane_cnt', 'hit_and_run_i'])

In [ ]:
crashes_in_areas

In [ ]:
## DROP any row with NaN values ** THIS REMOVES SOME CRASHES BUT KEEPS HIGH QUALITY RECORDED **
df = crashes_in_areas.dropna()
df

In [ ]:
## rename Community Area and Beat columns names
df = df.rename(columns = {"Community Area": "communityArea", "Beat": "beat"})
df

In [ ]:
df.dtypes

In [ ]:
# ## READ Chicago Area geoJSON file for merge
# chicagoAreas_df = gpd.read_file("static/data/chicago-community-areas.geojson")

In [ ]:
chicagoAreas_df
print(chicagoAreas_df)

In [ ]:
## REMOVE ** "area", "perimeter", "area_num_1", "comarea_id", "comarea"   ** from chicagoAreas_df
chicagoAreas_df = chicagoAreas_df.drop(columns=["area", "perimeter", "area_num_1", "comarea_id", "comarea"])
chicagoAreas_df = chicagoAreas_df.rename(columns = {"area_numbe": "communityArea"})
chicagoAreas_df.dtypes

In [ ]:
chicagoAreas_df[["communityArea"]] = chicagoAreas_df[["communityArea"]].apply(pd.to_numeric)
chicagoAreas_df.dtypes

In [ ]:
## MERGE THE TWO FILES TO CREATE MASTER 2019 GEOJSON
chicagoCrashMerge = pd.merge(left=df2019, right=chicagoAreas_df, how='right', on=['communityArea'])
chicagoCrashMerge

In [ ]:
chicagoCrashMerge['crashes_per_area'] = chicagoCrashMerge.groupby(['communityArea'])['community'].transform('count')
chicagoCrashMerge

In [ ]:
# CREATING GEOJSON FILE FOR EXPORT

## USING THE GEOPANDAS
# gdf_crashes2019 = geopandas.GeoDataFrame(
#     df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))
gdf_crashes2019 = gpd.GeoDataFrame(chicagoCrashMerge)

In [ ]:
print(gdf_crashes2019.head())

In [ ]:
ax = gdf_crashes2019.dropna().plot(column='crashes_per_area', cmap =    
                                'YlGnBu', figsize=(15,9),   
                                 scheme='quantiles', k=3, legend =  
                                  True);
#add title to the map
ax.set_title('Number of Crashes Per Chicago Area', fontdict= 
            {'fontsize':25})
#remove axes
ax.set_axis_off()
#move legend to an empty space
ax.get_legend().set_bbox_to_anchor((.12,.12))
ax.get_figure()

In [ ]:
#gdf_crashes2019.to_file("crashes_in_2019.geojson", driver='GeoJSON')